# ACF -- Ackerberg, Caves, Frazer
* This time, trying joint estimation
* Refer to ACF_simple_twostep for an easier version, which estimates $\Phi$ first with polynomial regression, then estimates the betas with GMM. 

# Review -- Two-Step Identification Procedure:
For a simple example, suppose $\omega_{it} = \rho \omega_{it-1} + \xi_{it}.$ Then
$g(x) = E[x|\omega_{it-1}] = \rho \omega_{it-1}.$ Assume labor is chosen after time $t-1$. Then the estimation procedure is: 

## (1) Regress $y_{it}$ on $\left(k_{it}, l_{it}, m_{it}\right)$ nonparametrically, or using a high-order polynomial, to obtain $\hat{\tilde \Phi}_t\left(k_{it}, l_{it}, m_{it}\right).$

We do this for every period to get a sequence of functions of $(k, l, m).$ These will be plugged in for $\Phi$ in the next step. 

## (2) Use the following four moment conditions to estimate the parameters $\left(\beta_0, \beta_k, \beta_l, \rho\right):$

$$
E\left[\left(y_{it} - \beta_0 - \beta_k k_{it} - \beta_l l_{it} - \rho\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right) \otimes \begin{pmatrix} 1 \\ k_{it} \\ l_{it-1} \\ \tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) \\ \end{pmatrix} \right] = 0
$$

Here's where we use GMM. 

# New version: Joint Identification

Now, we include the parameters used to fit $\Phi$ into the GMM estimation. 
We fit $\Phi$ with a $d$-degree polynomial, and the coefficients of that polynomial are identified by moments.
In ACF, equation (31) shows their joint estimation moment conditions: 

$$
    E\begin{bmatrix}
    \varepsilon_{it} \big| \mathcal I_{it} \\ 
    \xi_{it}+\varepsilon_{it} \big| \mathcal I_{it-1}                
    \end{bmatrix} = 
    E\begin{bmatrix}
    y_{it} - \tilde\Phi_t(k_{it}, l_{it}, m_{it}) \;\; \big| \;\; \mathcal I_{it} \\ 
    y_{it} - \beta_0 - \beta_k k_{it} - \beta_l l_{it} - g\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right) \;\; \big| \;\; \mathcal I_{it-1}                
    \end{bmatrix} 
$$

where $g(\cdot)$ is the conditional expectation of productivity and $\mathcal I_{it}$ is the information set of firm $i$ at time $t$.  

For each time t, the full $d$-degree polynomial fit of $y = \Phi_t(k, l, m)$ will have $\begin{pmatrix} k+d \\ k \end{pmatrix}$ coefficients, where $k$ is the number of variables (in this case 3). 

$$
\Phi_t^2(k, l, m) = \gamma_{0,0,0} + \gamma_{1,0,0}k + \gamma_{0,1,0}l + \gamma_{0,0,1}m + \gamma_{1,1,0}kl + \gamma_{1,0,1}km + \gamma_{0,1,1}lm + \gamma_{2,0,0}k^2 + \gamma_{0,2,0}l^2 + \gamma_{0,0,2}m^2
$$

But we need to fit $\Phi_t$ for each year. 

So, assuming $g(x) = \rho x$, the vector of parameters to identify is: 

$$
\mathbf \theta = \left[ \beta_0, \beta_k, \beta_l, \rho, \underbrace{\mathbf \gamma_{1}}_{kCd\times 1}, ...,  \underbrace{\mathbf \gamma_{T}}_{kCd\times 1}  \right]_{4 + T\cdot(kCd)}
$$

The first four parameters can be identified using the moments from the "simple" ACF:  

$$
E\left[\operatorname{vec}\left(y_{it} - \beta_0 - \beta_k k_{it} - \beta_l l_{it} - \rho\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right) \otimes \begin{pmatrix} 1 \\ k_{it} \\ l_{it-1} \\ \tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) \\ \end{pmatrix} \right] = 0
$$

Note that we will need to plug in the polynomial coefficients into the $\Phi$ here, from our parameter guess $\theta$. Also note that the final term with $\Phi$ in it is used as our moment for identifying $\rho$.

The $T\cdot(kCd)$ "gammas" can be estimated using the moments for linear regression, at each time period. Each of the vectors below is "Number of firms at time t"-by-1.

$$
\text{Regression at t=0:}\begin{pmatrix}
\mathbf 1^\intercal \left[ y_{i0} - \tilde\Phi_0(k_{i0}, l_{i0}, m_{i0})  \right] \\
 (k_{i0})^\intercal \left[ y_{i0} - \tilde\Phi_0(k_{i0}, l_{i0}, m_{i0})  \right] \\
 (l_{i0})^\intercal \left[ y_{i0} - \tilde\Phi_0(k_{i0}, l_{i0}, m_{i0})  \right] \\
 (m_{i0})^\intercal \left[ y_{i0} - \tilde\Phi_0(k_{i0}, l_{i0}, m_{i0})  \right] \\
(k_{i0}l_{i0})^\intercal \left[ y_{i0} - \tilde\Phi_0(k_{i0}, l_{i0}, m_{i0})  \right] \\
\vdots \\
 (l_{i0}^2)^\intercal \left[ y_{i0} - \tilde\Phi_0(k_{i0}, l_{i0}, m_{i0})  \right] \\
\end{pmatrix}
$$

$$
\text{Regression at t=T:}\begin{pmatrix}
\mathbf 1^\intercal \left[ y_{iT} - \tilde\Phi_T(k_{iT}, l_{iT}, m_{iT})  \right] \\
 (k_{iT})^\intercal \left[ y_{iT} - \tilde\Phi_T(k_{iT}, l_{iT}, m_{iT})  \right] \\
 (l_{iT})^\intercal \left[ y_{iT} - \tilde\Phi_T(k_{iT}, l_{iT}, m_{iT})  \right] \\
 (m_{iT})^\intercal \left[ y_{iT} - \tilde\Phi_T(k_{iT}, l_{iT}, m_{iT})  \right] \\
(k_{iT}l_{iT})^\intercal \left[ y_{iT} - \tilde\Phi_T(k_{iT}, l_{iT}, m_{iT})  \right] \\
\vdots \\
 (l_{iT}^2)^\intercal \left[ y_{iT} - \tilde\Phi_T(k_{iT}, l_{iT}, m_{iT})  \right] \\
\end{pmatrix}
$$
Here we have $T\cdot(kCd)$ moments fo the $T\cdot(kCd)$ "gammas." 

# Error function implementation. 
Let $X$ denote the polynomial regression design matrix and $\vec t$ denote the vector of times corresponding to each row in $X$. 

First, let $g(\tau): \;\text{time}\;\; \tau \to [\gamma_\tau^1, \gamma_\tau^l, ..., \gamma_\tau^{(M^2)}]$ denote the "time mapping" for the gammas. Using this mapping, construct 
a matrix $\Gamma = g(\vec t)$, whose rows correspond to the gammas associated with time of that each row's observation in  $X$. 

Then, $\Phi = \operatorname{row sum}\left(X*\Gamma\right)$, where $*$ denotes elementwise multiplication.  

Then, to calculate the errors, let $M_\tau$ denote a matrix of time dummies:

$$
M_\tau = \left[t==1 \big| t==2 \big| ... \big| t == T\right]_{Nobs \times T}
$$

Then the GMM errors for the regression moments can be calculated using 

$$
\mathcal E_{\mathcal N_p\times T } = X^\intercal\left[\Phi * M_\tau \right]
$$


# Jacobian Implementation: 
The Jacobian of the regression errros with respect to $\theta$ is: 

$$
\begin{bmatrix}
    0_{\beta} & \begin{matrix}
    1^\intercal\frac{\partial\Phi_0}{\partial \gamma_0^1} & \cdots & 1^\intercal\frac{\partial\Phi_0}{\partial \gamma_0^{(m_0^2)}} \\
    \vdots & \ddots & \vdots \\
    (m_0^2)^\intercal\frac{\partial\Phi_0}{\partial \gamma_0^1} & \cdots & (m_0^2)^\intercal\frac{\partial\Phi_0}{\partial \gamma_0^{(m_0^2)}}
    \end{matrix} & 0 & \cdots & 0 \\  0_{\beta} &
    0 &     \begin{matrix}
    1^\intercal\frac{\partial\Phi_1}{\partial \gamma_1^1} & \cdots & 1^\intercal\frac{\partial\Phi_1}{\partial \gamma_1^{(m_1^2)}} \\
    \vdots & \ddots & \vdots \\
    (m_1^2)^\intercal\frac{\partial\Phi_1}{\partial \gamma_1^1} & \cdots & (m_1^2)^\intercal\frac{\partial\Phi_1}{\partial \gamma_1^{(m_1^2)}}
    \end{matrix} & \cdots & 0 \\  \vdots &
    \vdots & \vdots & \ddots & \vdots \\  0_{\beta} &
    0 & 0 & \cdots &  \begin{matrix}
    1^\intercal\frac{\partial\Phi_T}{\partial \gamma_T^T} & \cdots & 1^\intercal\frac{\partial\Phi_T}{\partial \gamma_T^{(m_T^2)}} \\
    \vdots & \ddots & \vdots \\
    (m_T^2)^\intercal\frac{\partial\Phi_T}{\partial \gamma_T^1} & \cdots & (m_T^2)^\intercal\frac{\partial\Phi_T}{\partial \gamma_T^{(m_T^2)}}
    \end{matrix}
\end{bmatrix}
$$

Evaluating all the partial derivatives, we get

$$
\begin{bmatrix}
    0_{\beta} & X(1)^\intercal X(1) & 0 & \cdots & 0 \\  0_{\beta} &
    0 &     X(2)^\intercal X(2) & \cdots & 0 \\  \vdots &
    \vdots & \vdots & \ddots & \vdots \\  0_{\beta} &
    0 & 0 & \cdots &   X(T)^\intercal X(T)
\end{bmatrix}
$$

where $X(\tau)$ represents the observations of the polynomial regressors in $X$ at time $t = \tau$. 

Next, let's find the Jacobian of the "beta" moment restrictions. The conditions are
$$
\left[ \begin{pmatrix} 1^\intercal \\ \operatorname{vec}(k_{it})^\intercal \\ \operatorname{vec}(l_{it-1})^\intercal \\ \operatorname{vec}\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right)\right)^\intercal \\ \end{pmatrix} \cdot  h(\theta, \mathbf y, \mathbf k, \mathbf l)  \right]_{4\times 1}
$$
where 
$$
h(\theta, \mathbf y, \mathbf k, \mathbf l) = \operatorname{vec}\left(y_{it} - \beta_0 - \beta_k k_{it} - \beta_l l_{it} - \rho\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right).
$$
The Jacobian is: 

$$
\left[\frac{\partial e\left(\mathbf y, \mathbf k, \mathbf l|\theta\right)}{\partial \theta}\right] \equiv
$$




$$
= \begin{bmatrix} \begin{bmatrix}
1^\intercal\frac{ \mathbf \partial h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \beta_0} & 1^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \beta_k} & 1^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \beta_l} & 1^\intercal\frac{ \mathbf \partial h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \rho} \\
\operatorname{vec}(k_{it})^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_0} &\operatorname{vec}(k_{it})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_k} &\operatorname{vec}(k_{it})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_l} &\operatorname{vec}(k_{it})^\intercal  \frac{\partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \rho} \\
\operatorname{vec}(l_{it-1})^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_0} &\operatorname{vec}(l_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_k} &\operatorname{vec}(l_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_l} &\operatorname{vec}(l_{it-1})^\intercal  \frac{\partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \rho} \\
\operatorname{vec}(\Phi_{it-1})^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_0} &\operatorname{vec}(\Phi_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_k} &\operatorname{vec}(\Phi_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_l} &\operatorname{vec}(\Phi_{it-1})^\intercal  \frac{\partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \rho} 
\end{bmatrix} & 
\begin{bmatrix}
1^\intercal    \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_0^1} & \cdots & 1^\intercal    \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_T^{(m_T^2)}} \\
\operatorname{vec}(k_{it})^\intercal   \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_0^1} & \cdots & \operatorname{vec}(k_{it})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_T^{(m_T^2)}}  \\
\operatorname{vec}(l_{it-1})^\intercal   \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_0^1} & \cdots & \operatorname{vec}(l_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_T^{(m_T^2)}} \\ 
\operatorname{vec}\left(\frac{\partial\Phi_{it-1}}{\partial \gamma_0^1}\right)^\intercal \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l) + \operatorname{vec}(\Phi_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_0^1} & \cdots & \operatorname{vec}\left(\frac{\partial\Phi_{it-1}}{\partial \gamma_T^{(m_T^2)}}\right)^\intercal \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l) + \operatorname{vec}(\Phi_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \mathbf \gamma_T^{(m_T^2)}}
\end{bmatrix}
\end{bmatrix} 
$$


$$
\begin{bmatrix} \begin{bmatrix} 1^\intercal \\ \operatorname{vec}(k_{it})^\intercal \\ \operatorname{vec}(l_{it-1})^\intercal  \\ \operatorname{vec}(\Phi_{it-1})^\intercal \end{bmatrix}_{4\times n} \times \begin{bmatrix} \frac{\partial \mathbf h}{\partial \beta_0} & \frac{\partial \mathbf h}{\partial \beta_k} & \frac{\partial \mathbf h}{\partial \beta_l} & \frac{\partial \mathbf h}{\partial \rho} & \frac{\partial \mathbf h}{\partial \gamma_0^1} & \frac{\partial \mathbf h}{\partial \gamma_0^k} & \frac{\partial \mathbf h}{\partial \gamma_0^l} & \cdots & \frac{\partial \mathbf h}{\partial \gamma_T^{(m^2)}}  \end{bmatrix}_{nx(4+T*\mathcal N_p)}
\end{bmatrix}  
$$


# Calculating the derivatives of h:

$$
\mathbf h = \operatorname{vec}\left(y_{it} - \beta_0 - \beta_k k_{it} - \beta_l l_{it} - \rho\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right)
$$

$$
\frac{\partial \mathbf h}{\partial \beta_0} =  -1 + \rho
$$
$$
\frac{\partial \mathbf h}{\partial \beta_k} = -\operatorname{vec}(k_{it}) + \rho\operatorname{vec}(k_{it-1})
$$
$$
\frac{\partial \mathbf h}{\partial \beta_l} = -\operatorname{vec}(l_{it}) + \rho\operatorname{vec}(l_{it-1})
$$
$$
\frac{\partial \mathbf h}{\partial \rho} = \operatorname{vec}\left( - \left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right) 
$$
$$
\frac{\partial \mathbf h}{\partial \gamma_0^1} = -\rho \frac{\partial \mathbf \Phi_\text{prev}}{\partial \gamma_0^1} = 1 * \mathbf 1\left( t_\text{prev} == 0 \right)
$$
$$
\frac{\partial \mathbf h}{\partial \gamma_0^k} = -\rho \frac{\partial \mathbf \Phi_\text{prev}}{\partial \gamma_0^1} = k_\text{prev} * \mathbf 1\left( t_\text{prev} == 0 \right)
$$
... for arbitrary polynomial term $x$ and time $\tau$... 
$$
\frac{\partial \mathbf h}{\partial \gamma_\tau^x} = -\rho \frac{\partial \mathbf \Phi_\text{prev}}{\partial \gamma_0^1} = x_\text{prev} * \mathbf 1\left( t_\text{prev} == \tau \right)
$$


$$
\begin{bmatrix} \begin{bmatrix} 1^\intercal \\ \operatorname{vec}(k_{it})^\intercal \\ \operatorname{vec}(l_{it-1})^\intercal  \\ \operatorname{vec}(\Phi_{it-1})^\intercal \end{bmatrix}_{4\times n} \times 
\begin{bmatrix} 
\frac{\partial \mathbf h}{\partial \beta_0} & \frac{\partial \mathbf h}{\partial \beta_k} & \frac{\partial \mathbf h}{\partial \beta_l} & \frac{\partial \mathbf h}{\partial \rho} & -\rho 1 * \mathbf 1\left( t_\text{prev} == 0 \right) & -\rho k_\text{prev} * \mathbf 1\left( t_\text{prev} == 0 \right) & \cdots & -\rho x_\text{prev} * \mathbf 1\left( t_\text{prev} == \tau \right) & \cdots &  -\rho m^2_\text{prev}*\mathbf 1\left(t_\text{prev} == T\right) \end{bmatrix}_{n\times(4+T*\mathcal N_p)}
\end{bmatrix}  
$$

$$
+  \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)^\intercal \begin{bmatrix} 
0 & 0 & 0 & 0 &  1 * \mathbf 1\left( t_\text{prev} == 0 \right) &  k_\text{prev} * \mathbf 1\left( t_\text{prev} == 0 \right) & \cdots &  x_\text{prev} * \mathbf 1\left( t_\text{prev} == \tau \right) & \cdots &   m^2_\text{prev}*\mathbf 1\left(t_\text{prev} == T\right) \end{bmatrix}
$$


We can impement this part of the Jacobian by creating a matrix of time dummies for $tprev == t$ then multiplying the polynomial design matrix constructed of $kprev, lprev, mprev$. 

Let $M_{t_{prev}}$ be the matrix of time dummies which, for all times $\tau$, tells us which observations are at $tprev == \tau$.  The part with the regression moments is then 

$$
-\rho X_{prev} * \begin{bmatrix}
 \mathbf 1 \left\{t_{prev}==0\right\} & \mathbf 1 \left\{t_{prev}==1\right\} & \cdots & \mathbf 1 \left\{t_{prev}==T\right\} 
\end{bmatrix}
$$


# Load in the data


In [45]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt
import math
from itertools import combinations_with_replacement, chain

In [46]:
#Data transformation/utility functions
def df_to_dict(df):
    dict_of_arrays = {col: df[col].to_numpy() for col in df.columns}
    return dict_of_arrays

In [52]:
filename = "../PS3_data_changedtoxlsx.xlsx"
cols_to_keep = [0, 2, 3, 4, 5, 6, 40, 43, 44]
#new_names = ["year", "firm_id", "obs", "ly", "s01", "s02", "lc", "ll", "lm"]
new_names = ["t", "firm_id", "obs", "y", "s01", "s02", "k", "l", "m"]
#Load in the data
df = pd.read_excel(filename, usecols=cols_to_keep)
df.columns = new_names
#Keep industry 1 only
df=df[df['s01']==1]
#Creating lagged variables
df = df.sort_values(by=['firm_id', 't'])
df['kprev'] = df.groupby('firm_id')['k'].shift(1)
df['lprev'] = df.groupby('firm_id')['l'].shift(1)
df['mprev'] = df.groupby('firm_id')['m'].shift(1)

# First step of coding: Write functions for the estimation of $\tilde \Phi_t.$

In [120]:
#This thing creates an iterator structure of tuples, used to create 
#the design matrix of polynomial interaction terms. 
def poly_terms(n_features, degree):
    #It looks something like this
    #(0,), (1,), (2,), (0, 0), (0, 1), etc 
    polynomial_terms = chain(
        *(combinations_with_replacement(range(n_features), d) for d in range(1, degree+1))
    )
    return(polynomial_terms)

#Create design matrix for the polynomial fit
def poly_design_matrix(xvars, degree):
    #In case there's only one variable to fit
    if xvars.ndim == 1:
        xvars = xvars.reshape(1, -1)
    # Get the number of samples (n) and number of features (m) from X
    n_samples, n_features = xvars.shape
    # Create polynomial terms iterator
    polynomial_terms = poly_terms(n_features, degree)
    # Start with a column of ones for the intercept term
    X_poly = np.ones((n_samples, 1))
    # Generate polynomial terms and interaction terms up to 4th degree
    for terms in  polynomial_terms:  # For degrees 1 to 4
            X_poly = np.hstack((X_poly, np.prod(xvars[:, terms], axis=1).reshape(-1, 1)))
    return X_poly

#Create the matrix MT, which is used for year-by-year evaluation of the regression moments. 
def time_indexing(df, X_poly, degree = 2):
    #Times and indexing
    times = np.unique(df['t']) #get unique times in the sample
    T = len(times) #Number of unique times in the sample
    NPolyTerms= X_poly.shape[1] #Number of terms in the polynomial
    time_to_index = {time: idx for idx, time in enumerate(times)} # Create a dictionary mapping time values to the row index in G
    #Create a Nobs*T matrix of time dummies, used to create the error terms for GMM
    # Initialize the dummy matrix
    time_dummies = np.zeros((X_poly.shape[0], T))
    for i, t in enumerate(df['t']):
        time_index = np.where(times == t)[0][0]  # Find the index of the time in unique_times
        time_dummies[i, time_index] = 1
    return T, NPolyTerms, times, time_to_index, time_dummies   
    
#creates the first term in the moment restrictions:
def moment_error_ACF(theta, df, X_poly, degree = 2):
    Nobs = df.shape[0]
    #Get useful indexes
    T, NPolyTerms, times, time_to_index, time_dummies = time_indexing(df, X_poly, degree=2)
    #Reshape parameters theta. 
    betas = theta[:4] #extract beta_0, beta_k, beta_l, and rho
    gammas = theta[4:] #extract the guesses for the polynomial fit coefficeints (gammas).
    #Reshaope the gammas into a dictionary, indexed by time t of the observations. 
    Gamma_1_to_T = gammas.reshape(T, NPolyTerms) #This fills rows with K observations, then columns. 
    #Now, construct a mapping from GAMMA, whose rows are indexed by T, to the times in the sample
    GAMMA = np.vstack([Gamma_1_to_T[time_to_index[t]] for t in df['t']])
    #Now, evaluate Phi given the thetas.
    #To do this, elementwise-multiply the X with the polynomal coefficients in GAMMA. Sum the rows. 
    df['Phi'] = np.sum(X_poly*GAMMA, axis = 1)
    y_minus_Phi = df['y'] - df['Phi']
    PHI = np.repeat(y_minus_Phi.to_numpy()[:, np.newaxis], 10, axis=1)
    #Matrix of errors corresponding with regression moments (Nobs x T)
    Epsilon = (X_poly).T @ (PHI*time_dummies)
    #Lagged values of Phi
    df['Phiprev'] = df.groupby('firm_id')['Phi'].shift(1)
    #Now calculate the moment restrictions vector for the betas. 
    moments_betas = (df['y'] - theta[0] - theta[1]*df['k'] - theta[2]*df['l'] - 
             theta[3]*(df['Phiprev'] - theta[0] - theta[1]*df['kprev'] - theta[2]*df['lprev'] ) )
    #remove nans (associated with the lag) -- this is ok because we're just using this vector as part of a dot product. 
    moments_betas = np.nan_to_num(moments_betas, nan = 0)
    #Matrix of exclusion restrictions
    Vex = np.nan_to_num(np.vstack([
        np.ones(Nobs), 
        df['k'].to_numpy(), 
        df['lprev'].to_numpy(),
        df['Phiprev'].to_numpy()
    ]),  nan=0)
    #Evaluate the errors
    err_betas = Vex@moments_betas
    #Put together all of the errors into a single vector. 
    #Here, we want to unravel the Epsilon matrix so gammas are sorted into vectors by t. 
    epsilons = Epsilon.reshape(Epsilon.size)
    error_vec = np.concatenate((err_betas, epsilons))
    return error_vec

def gmm_obj_ACF(theta, df, X_poly, W):
    #Arguments
    #Get the vector h(theta, y, k, l)
    moment_error = moment_error_ACF(theta, df, X_poly, degree)
    #Calculate the weighted sum of the error using the weight matrix, W
    obj = moment_error.T@W@moment_error
    return obj

def jac_ACF(theta, df, X_poly, W):
    #Arguments
    xvars_prev = df[['kprev', 'lprev', 'mprev']].to_numpy()
    X_poly = poly_design_matrix(xvars, degree)
    #Get the vector h(theta, y, k, l)
    moment_error = moment_error_ACF(theta, df, X_poly, degree)
    #Calculate the weighted sum of the error using the weight matrix, W
    obj = moment_error.T@W@moment_error
    return obj


In [121]:
#Create polynomial design matrix
degree = 2
xvars = df[['k', 'l', 'm']].to_numpy()
X_poly = poly_design_matrix(xvars, degree)
#Get time dimension and number of polynomial terms 
T, NPolyTerms, times, time_to_index, time_dummies  = time_indexing(df, X_poly, degree)
#Initial guess for theta -- size depends on the degree of the polynomial and the number of times (number of phi_t fits)
theta0= np.ones(4+T*NPolyTerms)
W0 = np.eye(theta0.size)
#get error vector
error_vec = moment_error_ACF(theta0, df, X_poly, degree)
#Get gmm objective function error
error_obj = gmm_obj_ACF(theta0, df, X_poly, W0)

## Next step --  need to calculate the gradient to perform optimization. 

The gradient of the GMM objective function with respect to the parameters $\theta$ is
$$
\nabla_{\theta}  \mathbf e\left(\mathbf y, \mathbf k, \mathbf l|\theta\right)^\intercal_{1\times 4} \times\left(\mathbb W_{4\times 4}\right) \times      \mathbf e\left(\mathbf y, \mathbf k, \mathbf l|\theta\right)_{4\times 1} \equiv 2\left[\frac{\partial e\left(\mathbf y, \mathbf k, \mathbf l|\theta\right)}{\partial \theta}\right]^\intercal_{4\times 4}\times\left(\mathbb W_{4\times 4}\right) \times      \mathbf e\left(\mathbf y, \mathbf k, \mathbf l|\theta\right)_{4\times 1}
$$

where 

$$
\left[\frac{\partial e\left(\mathbf y, \mathbf k, \mathbf l|\theta\right)}{\partial \theta}\right]_{4\times 4} = \begin{bmatrix} 
1^\intercal\frac{ \mathbf \partial h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \beta_0} & 1^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \beta_k} & 1^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \beta_l} & 1^\intercal\frac{ \mathbf \partial h(\theta, \mathbf y, \mathbf k, \mathbf l)}{\partial \rho} \\
\operatorname{vec}(k_{it})^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_0} &\operatorname{vec}(k_{it})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_k} &\operatorname{vec}(k_{it})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_l} &\operatorname{vec}(k_{it})^\intercal  \frac{\partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \rho} \\
\operatorname{vec}(l_{it-1})^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_0} &\operatorname{vec}(l_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_k} &\operatorname{vec}(l_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_l} &\operatorname{vec}(l_{it-1})^\intercal  \frac{\partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \rho} \\
\operatorname{vec}(\Phi_{it-1})^\intercal\frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_0} &\operatorname{vec}(\Phi_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_k} &\operatorname{vec}(\Phi_{it-1})^\intercal \frac{ \partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \beta_l} &\operatorname{vec}(\Phi_{it-1})^\intercal  \frac{\partial \mathbf h(\theta, \mathbf y, \mathbf k, \mathbf l)
}{\partial \rho} 
\end{bmatrix} 
$$

$$
\equiv \begin{bmatrix} 1^\intercal \\ \operatorname{vec}(k_{it})^\intercal \\ \operatorname{vec}(l_{it-1})^\intercal  \\ \operatorname{vec}(\Phi_{it-1})^\intercal \end{bmatrix}_{4\times n} \begin{bmatrix} \frac{\partial \mathbf h}{\partial \beta_0} & \frac{\partial \mathbf h}{\partial \beta_k} & \frac{\partial \mathbf h}{\partial \beta_l} & \frac{\partial \mathbf h}{\partial \rho}  \end{bmatrix}_{4x4}
$$


is the Jacobian of the error function with respect to $\theta = \left(\beta_0, \beta_k, \beta_l, \rho\right)$. 

We already have code to calculate the first matrix here -- it is just called ```Vex```, "vectors of exogeneity restrictions."
It remains to calculate the partial derivatives in the second matrix.

$$
\mathbf h = \operatorname{vec}\left(y_{it} - \beta_0 - \beta_k k_{it} - \beta_l l_{it} - \rho\left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right)
$$

$$
\frac{\partial \mathbf h}{\partial \beta_0} =  -1 + \rho
$$
$$
\frac{\partial \mathbf h}{\partial \beta_k} = -\operatorname{vec}(k_{it}) + \rho\operatorname{vec}(k_{it-1})
$$
$$
\frac{\partial \mathbf h}{\partial \beta_l} = -\operatorname{vec}(l_{it}) + \rho\operatorname{vec}(l_{it-1})
$$
$$
\frac{\partial \mathbf h}{\partial \rho} = \operatorname{vec}\left( - \left(\tilde \Phi_{t-1}\left(k_{it-1}, l_{it-1}, m_{it-1}\right) - \beta_0 - \beta_k k_{it-1} - \beta_l l_{it-1}\right)\right) 
$$





In [18]:
def jacobian_ACF(theta, args, Vex):
    y, Phi, Phiprev, k, kprev, l, lprev = args
    
    #Partial derivatives of h
    Dh = np.nan_to_num(np.vstack(
        [
         np.ones(len(k))*(-1 + theta[3]), #dh/dbeta0  
         -k + theta[3]*kprev,             #dh/dbetak
         -l + theta[3]*lprev,             #dh/dbetal
         -(Phiprev - theta[0] - theta[1]*kprev - theta[2]*lprev)
        ]
    ),  nan=0).T

    Jac = Vex@Dh
    return Jac


def gradient_ACF(theta, args, Vex, W):
    moment_error = moment_error_ACF(theta, args)
    err = Vex@moment_error
    Jac = jacobian_ACF(theta, args, Vex)
    Grad = (2*Jac.T @ W @ err)
    return Grad


In [23]:
#testing
Jac = jacobian_ACF(theta0, args_ACF, Vex)
Jac
Grad = gradient_ACF(theta0, args_ACF, Vex, W0)
Grad

array([ 0.00000000e+00,  4.21897327e+06, -1.15537652e+04, -2.73682493e+07])

## Now, use a minimization routine, with the Jacobian, to optimize for theta. 

In [48]:
args_ACF = format_args_ACF(df)
gmm_args = (args_ACF, Vex, W0)

#Solving using my own gradient
theta_results_grad = opt.minimize(gmm_obj_ACF, theta0, args=gmm_args,
                       tol=1e-14, jac=gradient_ACF, method='L-BFGS-B')
#Solving without providing a gradient
theta_results_nograd = opt.minimize(gmm_obj_ACF, theta0, args=gmm_args,
                        tol=1e-14, method='L-BFGS-B')
theta_results_grad.x, theta_results_nograd.x
theta_results = theta_results_grad.x

gmm_obj_ACF(theta_results, args_ACF, Vex, W0)

0.010508466106636766